[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=SimpleMarketmaker.ipynb) [Run this code](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=SimpleMarketmaker.ipynb) on Binder.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Simple Marketmaker

This notebook shows how to run a simple marketmaker on a market.

Marketmakers typically place a pair of BUY and a SELL orders, hoping (on average) to profit by capturing the spread between those prices. `mango-explorer` has a [short introduction to marketmaking](https://github.com/blockworks-foundation/mango-explorer/blob/main/docs/MarketmakingIntroduction.md) that starts with a very basic marketmaker (a `bash` script!) and increases the complexity to a more complete marketmaker bot.

That bot has a lot of [configuration options](https://github.com/blockworks-foundation/mango-explorer/blob/main/docs/MarketmakingOrderChain.md) and a useful [Quickstart](https://github.com/blockworks-foundation/mango-explorer/blob/main/docs/MarketmakingQuickstart.md) to get you up and running marketmaking on devnet.

This code is simpler, intending to demonstrate the principlea of:
* placing paired orders at a specified distance from an oracle price,
* pausing (hoping the orders will be filled), then
* replacing those orders.

More sophisticated error handling, risk management, pricing, and inventory handling are available in `mango-explorer`.

_(Note: it's probably not a good idea to run this code for extended periods of time. Binder can time out, and the output log at the bottom of the page can grow to an extent that causes problems with the browser. If you want to run something this simple for an extended period of time, it may be better to check out the `simple-marketmaker` command in `mango-explorer` which follows the same principles as this code but is better suited to continuous operation.)_

In [ ]:
import decimal
import mango
import time

# Use our hard-coded devnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file.
wallet = mango.Wallet(bytes([67,218,68,118,140,171,228,222,8,29,48,61,255,114,49,226,239,89,151,110,29,136,149,118,97,189,163,8,23,88,246,35,187,241,107,226,47,155,40,162,3,222,98,203,176,230,34,49,45,8,253,77,136,241,34,4,80,227,234,174,103,11,124,146]))

# Create a 'devnet' Context
with mango.ContextBuilder.build(cluster_name="devnet") as context:
    # Load the wallet's account
    group = mango.Group.load(context)
    accounts = mango.Account.load_all_for_owner(context, wallet.address, group)
    account = accounts[0]

    # Load the market
    market = mango.market(context, "SOL-PERP")
    market_operations = mango.operations(context, wallet, account, "SOL-PERP", dry_run=False)

    oracle_provider = mango.create_oracle_provider(context, "pyth")
    sol_oracle = oracle_provider.oracle_for_market(context, market)

    # OK, that's the setup done. Now we're ready to loop.
    # Every 30 seconds (`pause_seconds`) the code will:
    # * Cancel all orders
    # * Place a BUY order for 1 SOL-PERP (`quantity`) at currentprice - 1% (`price_factor`)
    # * Place a SELL order for 1 SOL-PERP (`quantity`) at currentprice + 1% (`price_factor`)
    pause_seconds = 30
    quantity = decimal.Decimal(1)
    price_factor = decimal.Decimal("0.01")  # 1%, so orders will be at + and - 1% of oracle mid price.
    stop_requested = False
    while not stop_requested:
        try:
            # Update current price
            price = sol_oracle.fetch_price(context)
            print(f"Price is: ${price.mid_price:,.2f}")

            print("Cancelling orders.")
            for order in market_operations.load_my_orders():
                print(f"- Cancelling: {order}")
                market_operations.cancel_order(order, ok_if_missing=True)

            # Calculate what we want the orders to be.
            price_adjustment = price.mid_price * price_factor
            bid = price.mid_price - price_adjustment
            ask = price.mid_price + price_adjustment

            print("Placing orders.")
            buy_order = mango.Order.from_values(mango.Side.BUY, bid, quantity, mango.OrderType.POST_ONLY)
            placed_buy = market_operations.place_order(buy_order)
            print(f"+ {placed_buy}")

            sell_order = mango.Order.from_values(mango.Side.SELL, ask, quantity, mango.OrderType.POST_ONLY)
            placed_sell = market_operations.place_order(sell_order)
            print(f"+ {placed_sell}")

            # Wait and hope for fills.
            print(f"Pausing for {pause_seconds} seconds.\n")
            time.sleep(pause_seconds)
        except KeyboardInterrupt:
            stop_requested = True
        except Exception as exception:
            print(f"Continuing after problem running market-making iteration: {exception}")

    print("\n\nCancelling all orders and stopping...")
    for order in market_operations.load_my_orders():
        market_operations.cancel_order(order, ok_if_missing=True)

print("Example complete.")